### 进程与线程

#### 今天我们使用的计算机早已进入多CPU或多核时代，而我们使用的操作系统都是支持“多任务”的操作系统，这使得我们可以同时运行多个程序，也可以将一个程序分解为若干个相对独立的子任务，让多个子任务并发的执行，从而缩短程序的执行时间，同时也让用户获得更好的体验。因此在当下不管是用什么编程语言进行开发，实现让程序同时执行多个任务也就是常说的“并发编程”，应该是程序员必备技能之一。为此，我们需要先讨论两个概念，一个叫进程，一个叫线程。

### 概念

#### 进程就是操作系统中执行的一个程序，操作系统以进程为单位分配存储空间，每个进程都有自己的地址空间、数据栈以及其他用于跟踪进程执行的辅助数据，操作系统管理所有进程的执行，为它们合理的分配资源。进程可以通过fork或spawn的方式来创建新的进程来执行其他的任务，不过新的进程也有自己独立的内存空间，因此必须通过进程间通信机制（IPC，Inter-Process Communication）来实现数据共享，具体的方式包括管道、信号、套接字、共享内存区等。
#### 一个进程还可以拥有多个并发的执行线索，简单的说就是拥有多个可以获得CPU调度的执行单元，这就是所谓的线程。由于线程在同一个进程下，它们可以共享相同的上下文，因此相对于进程而言，线程间的信息共享和通信更加容易。当然在单核CPU系统中，真正的并发是不可能的，因为在某个时刻能够获得CPU的只有唯一的一个线程，多个线程共享了CPU的执行时间。使用多线程实现并发编程为程序带来的好处是不言而喻的，最主要的体现在提升程序的性能和改善用户体验，今天我们使用的软件几乎都用到了多线程技术
#### 当然，多线程也并不是没有坏处，站在其他进程的角度，多线程的程序对其他程序并不友好，因为它占用了更多的CPU执行时间，导致其他程序无法获得足够的CPU执行时间；另一个方面，站在开发者的角度。编写和调试多进程的程序都对开发者有较高的要求，对于初学者来说更加困难
#### python既支持多进程又支持多线程，因此可以使用python实现并发编程主要有3种方式：多进程、多线程、多进程+多线程

### fork

#### Unix/Linux操作系统提供了一个fork()系统调用，它非常特殊，普通的函数调用，调用一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称之为父进程）复制了一份（称之为子进程），然后在父进程和子进程内返回。
#### 子进程永远返回0，二父进程返回子进程的ID，这样做的理由是一个父进程可以fork出很多的子进程，所以父进程要记住每一个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID

In [4]:
# mac上的多进程
import os

print ("Process (%s) start ..." % os.getpid())
pid = os.fork()
if pid ==0:
    print("I am child process (%s) and my parent is %s." %(os.getpid(),os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(),pid))

In [6]:
#windows上的多进程
from multiprocessing import Process
import os

#子进程要执行的代码
def run_proc(name):
    print("Run child process %s (%s)" % (name,os.getpid()))
    
if __name__ == "__main__":
    print("Parent process %s." % os.getpid())
    p = Process(target=run_proc,args = ('test',))
    print("Process will start.")
    p.start()
    p.join()
    print("Process end.")

Parent process 6996.
Process will start.
Process end.


### python中的多线程

In [21]:
#没有使用多线程
from random import randint
from time import time,sleep

def download_task(filename):
    print('开始下载%s...'% filename)
    time_to_sleep = randint(5,10)
    sleep(time_to_sleep)
    print('%s下载完成！耗费时间%s秒' % (filename,time_to_sleep))
    
def main():
    start = time()
    download_task('《Python Cookbook》第三版中文v2.0.0.pdf')
    download_task('第13章 多线程与多进程编程.ppt')
    end = time()
    print('总耗费了%.2f秒'%(end-start))
    
if __name__ =="__main__":
    main()

开始下载《Python Cookbook》第三版中文v2.0.0.pdf...
《Python Cookbook》第三版中文v2.0.0.pdf下载完成！耗费时间5秒
开始下载第13章 多线程与多进程编程.ppt...
第13章 多线程与多进程编程.ppt下载完成！耗费时间7秒
总耗费了12.00秒


#### 从上面的例子可以看出，如果程序中的代码只能按顺序一点点的往下执行，那么即使执行两个毫不相关的下载任务，也需要先等待一个文件下载完成后才能开始下一个下载任务，很显然这并不合理也没有效率。接下来我们使用多进程的方式将两个下载任务放到不同的进程中，代码如下所示。

In [ ]:
from multiprocessing import Process
from os import getpid
from random import randint
from time import time, sleep


def download_task(filename):
    print('启动下载进程，进程号[%d].' % getpid())
    print('开始下载%s...' % filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    p1 = Process(target=download_task, args=('《Python Cookbook》第三版中文v2.0.0.pdf', ))
    p1.start()
    p2 = Process(target=download_task, args=('第13章 多线程与多进程编程.ppt', ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))


if __name__ == '__main__':
    main()

In [ ]:
#因为多个线程可以共享进程的内存空间，因此要实现多个线程间的通信相对简单，大家能想到的最直接的办法就是设置一个全局变量，多个线程共享这个全局
#变量即可。但是当多个线程共享同一个变量（我们通常称之为“资源”）的时候，很有可能产生不可控的结果从而导致程序失效甚至崩溃。如果一个资源被多
#个线程竞争使用，那么我们通常称之为“临界资源”，对“临界资源”的访问需要加上保护，否则资源会处于“混乱”的状态。下面的例子演示了100个线程
#向同一个银行账户转账（转入1元钱）的场景，在这个例子中，银行账户就是一个临界资源，在没有保护的情况下我们很有可能会得到错误的结果。
from time import sleep
from threading import Thread


class Account(object):

    def __init__(self):
        self._balance = 0

    def deposit(self, money):
        # 计算存款后的余额
        new_balance = self._balance + money
        # 模拟受理存款业务需要0.01秒的时间
        sleep(0.01)
        # 修改账户余额
        self._balance = new_balance

    @property
    def balance(self):
        return self._balance


class AddMoneyThread(Thread):

    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money

    def run(self):
        self._account.deposit(self._money)


def main():
    account = Account()
    threads = []
    # 创建100个存款的线程向同一个账户中存钱
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    # 等所有存款的线程都执行完毕
    for t in threads:
        t.join()
    print('账户余额为: ￥%d元' % account.balance)


if __name__ == '__main__':
    main()

In [ ]:
#运行上面的程序，结果让人大跌眼镜，100个线程分别向账户中转入1元钱，结果居然远远小于100元。之所以出现这种情况是因为我们没有对银行账户这个
#“临界资源”加以保护，多个线程同时向账户中存钱时，会一起执行到new_balance = self._balance + money这行代码，多个线程得到的账户余额都是初始
#状态下的0，所以都是0上面做了+1的操作，因此得到了错误的结果。在这种情况下，“锁”就可以派上用场了。我们可以通过“锁”来保护“临界资源”，
#只有获得“锁”的线程才能访问“临界资源”，而其他没有得到“锁”的线程只能被阻塞起来，直到获得“锁”的线程释放了“锁”，其他线程才有机会获得
#“锁”，进而访问被保护的“临界资源”。下面的代码演示了如何使用“锁”来保护对银行账户的操作，从而获得正确的结果。
from time import sleep
from threading import Thread, Lock


class Account(object):

    def __init__(self):
        self._balance = 0
        self._lock = Lock()

    def deposit(self, money):
        # 先获取锁才能执行后续的代码
        self._lock.acquire()
        try:
            new_balance = self._balance + money
            sleep(0.01)
            self._balance = new_balance
        finally:
            # 在finally中执行释放锁的操作保证正常异常锁都能释放
            self._lock.release()

    @property
    def balance(self):
        return self._balance


class AddMoneyThread(Thread):

    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money

    def run(self):
        self._account.deposit(self._money)


def main():
    account = Account()
    threads = []
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('账户余额为: ￥%d元' % account.balance)


if __name__ == '__main__':
    main()

In [24]:
#练习：使用多进程对复杂的任务进行分而治之
from time import time 

def main():
    total = 0
    number_list = [x for x in range(1,100000001)]
    start_time = time()
    for number in number_list:
        total += number
    print(total)
    end =  time()
    print('Execution time: %.3f seconds' % (end - start_time))
    
if __name__ == "__main__":
    main()

5000000050000000
Execution time: 5.476 seconds


In [25]:
#在上面的代码中，我故意先去创建了一个列表容器然后填入了100000000个数，这一步其实是比较耗时间的，所以为了公平起见，当我们将这个任务分解到8个
#进程中去执行的时候，我们暂时也不考虑列表切片操作花费的时间，只是把做运算和合并运算结果的时间统计出来，代码如下所示。
from multiprocessing import Process,Queue
from random import randint
from time import time 

def task_handler(curr_list,result_queue):
    total = 0
    for number in curr_list:
        total += number
    result_queue.put(total)
    
def main():
    processes = []
    number_list = [x for x in range(1,100000001)]
    result_queue = Queue()
    index = 0
    #启动8个进程将数据切片后进行运算
    for _ in range(8):
        p = Process(target=task_handler,args = (number_list[index:index+12500000],result_queue))
        index += 12500000
        processes.append(p)
        p.start()
        #开始记录所有进程完成花费的时间
        start = time()
        for p in processes:
            p.join()
        #合并结果
        total = 0
        while not result_queue.empty():
            total += result_queue.get()
        print(total)
        end = time()
        print('Execution time:',(end-start),'s',sep='')
        
if __name__ == "__main__":
    main()

0
Execution time:1.2876603603363037s
0
Execution time:0.33911800384521484s
0
Execution time:0.3662109375s
0
Execution time:0.927131175994873s
0
Execution time:0.5117044448852539s
0
Execution time:1.1375408172607422s
0
Execution time:0.3889594078063965s
0
Execution time:0.3978588581085205s
